In [47]:
import pandas as pd
import numpy as np

In [48]:
qpd_df = pd.read_csv("data/exports/qinn_processed_data.csv")
df = qpd_df.copy()

In [49]:
df.sample()

,room_num,room_type,customer_id,stay_duration,total_price,price,tax,customer_name,checkin_datetime,checkout_datetime,...,checkin_weekday,checkin_day_name,checkin_time_period,checkout_month,checkout_day,checkout_year,checkout_time,checkout_weekday,checkout_day_name,checkout_time_period
1132,111,K,TX22351935,3,40.0,34.19,5.81,CATCHINGS LARISSA,2023-02-18 00:51:00,2023-02-18 03:51:00,...,5,Saturday,Night,2,18,2023,03:51:00,5,Saturday,Night


In [50]:
df.groupby(['checkin_year','checkin_month'])['total_price'].sum()

checkin_year  checkin_month
2022          10               13402.34
              11               24283.33
              12               31711.62
2023          1                21984.73
              2                17359.28
              3                21070.20
              4                23116.47
              5                20981.48
              6                34604.91
              7                40508.55
              8                36533.06
              9                30241.80
              10                3095.00
Name: total_price, dtype: float64

In [51]:
# Fill missing values with mean 
df['fill_df'] = df['total_price']
df['fill_df'] = df['fill_df'].replace(0.00, np.nan)

In [52]:
df['fill_p25'] = df.groupby(['checkin_year','checkin_month', 'room_type', 'stay_duration'])['fill_df'].transform(lambda x: x.fillna(x.quantile(.25)))
df['fill_p50'] = df.groupby(['checkin_year','checkin_month', 'room_type', 'stay_duration'])['fill_df'].transform(lambda x: x.fillna(x.quantile(.50)))
df['fill_p75'] = df.groupby(['checkin_year','checkin_month', 'room_type', 'stay_duration'])['fill_df'].transform(lambda x: x.fillna(x.quantile(.75)))
df['fill_p90'] = df.groupby(['checkin_year','checkin_month', 'room_type', 'stay_duration'])['fill_df'].transform(lambda x: x.fillna(x.quantile(.90)))
df['fill_p95'] = df.groupby(['checkin_year','checkin_month', 'room_type', 'stay_duration'])['fill_df'].transform(lambda x: x.fillna(x.quantile(.95)))
df['fill_p99'] = df.groupby(['checkin_year','checkin_month', 'room_type', 'stay_duration'])['fill_df'].transform(lambda x: x.fillna(x.quantile(.99)))

df.loc[df.total_price == 0.00, ['total_price', 'room_type', 'stay_duration', 'fill_p25', 'fill_p50', 'fill_p75', 'fill_p90', 'fill_p95', 'fill_p99']]

,total_price,room_type,stay_duration,fill_p25,fill_p50,fill_p75,fill_p90,fill_p95,fill_p99
12,0.0,NK,N,60.000,70.00,75.000,91.400,195.356,245.5000
42,0.0,K,N,70.000,70.00,70.000,74.560,85.000,108.8000
44,0.0,NK,N,51.675,70.00,70.000,85.000,85.000,88.3500
54,0.0,K,N,60.000,61.51,70.000,70.500,72.750,74.5500
57,0.0,K,N,67.500,70.00,70.000,121.692,177.700,204.3300
...,...,...,...,...,...,...,...,...,...
6465,0.0,K,N,50.000,55.00,65.000,74.000,75.000,77.6996
6549,0.0,NQQ,N,70.000,70.00,78.750,80.000,85.250,93.0500
6559,0.0,NK,N,70.000,70.00,75.850,85.500,90.000,90.0000
6561,0.0,K,N,50.000,61.20,70.000,75.000,76.250,95.2610


In [53]:
# Create new dataframe with adjusted values
adj_df = df.loc[df.total_price == 0.00, ['total_price', 'room_type', 'stay_duration', 'fill_p25', 'fill_p50', 'fill_p75', 'fill_p90', 'fill_p95', 'fill_p99']]

In [54]:
# Get the difference between the original and adjusted total price
delta = format(adj_df['fill_p25'].sum() - adj_df['total_price'].sum(), ".2f")
print(f"Total price difference: ${delta}")

Total price difference: $45944.13


In [55]:
# Kishivam Hospitalities LLC
df[(df.checkin_year == 2023)&(df.checkin_month >= 6)].groupby(['checkin_year','checkin_month'])['fill_p25'].sum()

checkin_year  checkin_month
2023          6                34844.91
              7                41177.40
              8                36698.06
              9                30301.80
              10                3095.00
Name: fill_p25, dtype: float64

In [56]:
# Group by 'checkin_year' and 'checkin_month' and calculate the sum of 'total_price' and 'fill_pn'
monthly_df = df[df.checkin_year == 2023].groupby(['checkin_year','checkin_month']).agg({'total_price': 'sum', 'fill_p25': 'sum'})

# Calculate the difference between the sums of each month for 'total_price' and 'fill_pn'
monthly_df['diff_sum'] = monthly_df['fill_p25'] - monthly_df['total_price']

monthly_df

total_price    fill_p25   diff_sum
checkin_year checkin_month                                    
2023         1                 21984.73  30012.6675  8027.9375
             2                 17359.28  23144.4800  5785.2000
             3                 21070.20  25622.5300  4552.3300
             4                 23116.47  29946.0825  6829.6125
             5                 20981.48  26531.4550  5549.9750
             6                 34604.91  34844.9100   240.0000
             7                 40508.55  41177.4000   668.8500
             8                 36533.06  36698.0600   165.0000
             9                 30241.80  30301.8000    60.0000
             10                 3095.00   3095.0000     0.0000